# Chapter 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 1.1
### 1.1.a

According to the law of total probability (LOTP) the marginal density of *y* is defined as the weighted average of the conditional densities:
$$p(y) = \sum_{\theta}{p(y|\theta) * p(\theta)}$$
In the present examples with two possible values for theta, this is
$$p(y) = p(y|\theta_1)*p(\theta_1)+p(y|\theta_2)*p(\theta_2)$$
Since both are normal and the thetas a equally likely unconditionally, the equation takes the following form:
$$p(y) = \frac{1}{2}*(\mathcal{N}(y|0, 2^2) + \mathcal{N}(y|1, 2^2))$$

In [ ]:
prior = np.array([.5, .5]) # prior on thetas
mus = [0, 1]
x = np.linspace(-4, 4)